In [2]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install umap-learn
!pip install hdbscan
!pip install sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import os
import csv
from google.colab import drive
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

Let google colab access drive

In [4]:
drive.mount('/content/drive')

# Check if the mount was successful
if os.path.exists('/content/drive/MyDrive'):
  print("Google Drive mounted successfully!")
else:
  print("Failed to mount Google Drive. Please check authentication and network.")

Mounted at /content/drive
Google Drive mounted successfully!


Set our input and output directories and check that gcolab can access them.

In [38]:
# input_directory = '/content/drive/My Drive/Where In The World/Segmentation/Segmentation OUTPUT'
# output_directory = '/content/drive/My Drive/Where In The World/Topic Modeling WITW/Topic Modeling OUTPUT'

input_directory = '/content/drive/My Drive/Where In The World/Segmentation/Segmentation OUTPUT/'
output_directory = '/content/drive/My Drive/Where In The World/Topic Modeling WITW/'

embeddings_directory = '/content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings'


# CHECK INPUT PATH IS VALID
if os.path.exists(input_directory):
  print(f"Input directory exists: {input_directory}")
  print("Contents of the directory:")
  print(os.listdir(input_directory))
else:
  print(f"Input directory does not exist: {input_directory}")

# CHECK OUTPUT PATH IS VALID
if os.path.exists(output_directory):
  print(f"Output directory exists: {output_directory}")
  print("Contents of the directory:")
  print(os.listdir(output_directory))
else:
  print(f"Output directory does not exist: {output_directory}")

Input directory exists: /content/drive/My Drive/Where In The World/Segmentation/Segmentation OUTPUT/
Contents of the directory:
['Batch_1', 'Batch_2', 'Batch_3', 'Batch_4', 'Batch_5', 'Batch_6', 'Batch_7', 'Batch_8', 'Batch_9', 'Batch_10', 'Batch_11', 'Batch_12']
Output directory exists: /content/drive/My Drive/Where In The World/Topic Modeling WITW/
Contents of the directory:
['embeddings', 'TRIAL_1_output.gsheet']


Go through our corpus and add each segment to a list. You have to manually change the directory each time--this is intentional, because the size of the corpus makes gcolab crash when trying to get the embeddings for the entire thing at once. Instead, we pre-calculate the embeddings in batches. So we will run the following code to read in documents for ONE batch, then run following cells to calculate and SAVE the embeddings for that batch. Then manually change the input_directory to the second batch of files, and so on.

In [ ]:
# reading the documents for topic modelling
input_directory = '/content/drive/My Drive/Where In The World/Segmentation/Segmentation OUTPUT/Batch_1'

documents = []

for csvfile in os.scandir(input_directory):
  if csvfile.is_file() and csvfile.name.endswith('.csv'):
    with open(csvfile.path, 'r', newline='', encoding='utf-8') as f:
      reader = csv.reader(f)
      for row in reader:
        # Assuming each row is a single segment/paragraph
        if row:  # Check if the row is not empty
          doc = str(row[0])  # Convert the first column to string
          documents.append(doc)

In [ ]:
print(len(documents))

11564


In [18]:
# You only need to run this cell once
embedding_model = SentenceTransformer('Lajavaness/bilingual-embedding-large', trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/242k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

(…)ipynb_checkpoints/config-checkpoint.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
embeddings = embedding_model.encode(documents)

In [ ]:
# CHANGE THE NAME of the output file to whatever batch was input
output_filename = 'Batch_12_embeddings.bin'
output_path = os.path.join(embeddings_directory, output_filename)

embeddings.tofile(output_path)

print(f"Embeddings saved to: {output_path}")

Embeddings saved to: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_12_embeddings.bin


Now we have pre-calculated the embeddings. The next step is to put it all back together, since we want to run topic modeling on the entire corpus at once. This means that: \\
1) we have to make a 'master list' of all the segments \\
2) we have to recombine all the embeddings, since they were saved in batches \\
3) IMPORTANT: the order of the embeddings MUST match the order in which the documents were read during the pre-calculation. Be very careful to make sure we put all the batches, for both the documents and the embeddings, IN ORDER when we read them all in.

In [29]:
# GET OUR MASTER LIST OF DOCUMENTS

input_directory = '/content/drive/My Drive/Where In The World/Segmentation/Segmentation OUTPUT/'

doc_directory_names = [
    'Batch_1',
    'Batch_2',
    'Batch_3',
    'Batch_4',
    'Batch_5',
    'Batch_6',
    'Batch_7',
    'Batch_8',
    'Batch_9',
    'Batch_10',
    'Batch_11',
    'Batch_12',
]

In [30]:
master_doclist = []

for doc_directory in doc_directory_names:
  file_path = os.path.join(input_directory, doc_directory)
  for csvfile in os.scandir(file_path):
    if csvfile.is_file() and csvfile.name.endswith('.csv'):
      with open(csvfile.path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
          # Assuming each row is a single segment/paragraph
          if row:  # Check if the row is not empty
            doc = str(row[0])  # Convert the first column to string
            master_doclist.append(doc)

In [31]:
print(len(master_doclist))

"""
Important note: there should be exactly the same amount of embeddings as there are documents/segments in the master list. We print the length of the master list to make sure this is the case.
"""

206867


In [33]:
# RECOMBINE THE EMBEDDINGS
embedding_dim = 1024

batch_filenames = [
    'Batch_1_embeddings.bin',
    'Batch_2_embeddings.bin',
    'Batch_3_embeddings.bin',
    'Batch_4_embeddings.bin',
    'Batch_5_embeddings.bin',
    'Batch_6_embeddings.bin',
    'Batch_7_embeddings.bin',
    'Batch_8_embeddings.bin',
    'Batch_9_embeddings.bin',
    'Batch_10_embeddings.bin',
    'Batch_11_embeddings.bin',
    'Batch_12_embeddings.bin',
]

# Initialize an empty list to store the loaded embeddings
all_embeddings = []

# Loop through the batch files and load each one
for filename in batch_filenames:
    file_path = os.path.join(embeddings_directory, filename)
    # Load the embeddings from the binary file
    curr_embeddings = np.fromfile(file_path, dtype=np.float32)

    # IMPORTANT: because we previously stored the embeddings in a flattened array, we need to reshape it.
    # Reshaping will make the num docs = num embeddings, as previously mentioned they have to be the exact same amount of docs:embeddings in order to run topic modeling
    # We reshape it in 1024 dimension because that is the dimension that the huggingface model uses.
    curr_embeddings = curr_embeddings.reshape(-1, embedding_dim)

    print(f"Loaded embeddings from: {file_path}")
    print("Shape of embeddings:", curr_embeddings.shape)

    all_embeddings.append(curr_embeddings)

# Concatenate all the embeddings into a single array
all_embeddings = np.concatenate(all_embeddings, axis=0)

# Verify the final shape of all_embeddings
print("Final shape of embeddings:", all_embeddings.shape)

Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_1_embeddings.bin
Shape of embeddings: (26143, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_2_embeddings.bin
Shape of embeddings: (40548, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_3_embeddings.bin
Shape of embeddings: (27771, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_4_embeddings.bin
Shape of embeddings: (26660, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_5_embeddings.bin
Shape of embeddings: (9113, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The World/Topic Modeling WITW/embeddings/Batch_6_embeddings.bin
Shape of embeddings: (16172, 1024)
Loaded embeddings from: /content/drive/My Drive/Where In The Worl

In [35]:
if len(master_doclist) != all_embeddings.shape[0]:
    raise ValueError(f"Number of documents ({len(master_doclist)}) does not match the number of embeddings ({all_embeddings.shape[0]}).")

Prepare the model pipeline components; now we initialize the other parts we need for topic modeling.

In [14]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [15]:
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [16]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop)

vectorizer_model = CountVectorizer(stop_words=final_stopwords_list, min_df=2, max_df=0.95, ngram_range=(1, 3))
# n-gram range = number of words a topic can be

Run topic modeling and save the results

In [36]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,

  # Hyperparameters
  top_n_words=30,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(master_doclist, all_embeddings)

# Show topics
topic_model.get_topic_info()

2024-08-26 21:03:11,536 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-26 21:10:20,551 - BERTopic - Dimensionality - Completed ✓
2024-08-26 21:10:20,560 - BERTopic - Cluster - Start clustering the reduced embeddings
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
2024-08-26 21:11:27,545 - BERTopic - Cluster - Completed ✓
2024-08-26 21:11:27,611 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-26 21:11:49,442 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,108642,-1_québec_montréal_bien_travail,"[québec, montréal, bien, travail, faire, 000, ...",[Rappelons que c'est en janvier 1980 que le Go...
1,0,8046,0_eglise_dieu_chrétiens_foi,"[eglise, dieu, chrétiens, foi, prêtres, christ...",[Ce peuple qui vit dans la répression est comp...
2,1,7992,1_québécois_parti_québec_français,"[québécois, parti, québec, français, langue, c...","[Cependant, Johnson oppose avec raison les mis..."
3,2,5998,2_temps_vie_bien_faut,"[temps, vie, bien, faut, monde, amour, sommes,...",[rayons du soleil; pour nous cette lumière est...
4,3,4218,3_enseignants_étudiants_professeurs_cours,"[enseignants, étudiants, professeurs, cours, é...","[Ainsi, sur la Rive-Sud de Montréal, dans le t..."
...,...,...,...,...,...
86,85,189,85_mètres_secondes_record_descente,"[mètres, secondes, record, descente, saut, sla...",[Nobukata Taguchi du Japon a égalé le record m...
87,86,180,86_sherbrooke_rue_inc_ltée,"[sherbrooke, rue, inc, ltée, 569, st, gr, tél,...",[1\t562-3857 L. LAVIGNE LTEE !\tEmbouleilleur ...
88,87,178,87_congrès_congres_délégués_résolutions,"[congrès, congres, délégués, résolutions, cong...",[La campagne électorale aura marqué profondéme...
89,88,165,88_adresse_nom adresse_adresses_droite rue,"[adresse, nom adresse, adresses, droite rue, h...",[Adresse.........................................


In [39]:
# Save the output of the topic modeling to a readable file

topic_info_df = topic_model.get_topic_info()

output_csv_path = os.path.join(output_directory, "topic_info.csv")
topic_info_df.to_csv(output_csv_path, index=False)  # index=False to avoid saving the DataFrame index

print(f"Topic information saved to {output_csv_path}")

Topic information saved to /content/drive/My Drive/Where In The World/Topic Modeling WITW/topic_info.csv


CODE BELOW COPIED FROM PARTI PRIS TOPIC MODELING / not yet relevant to this project


In [ ]:
# Create a mapping of text segments to their topics and probabilities
topic_mapping = []
for doc, topic, prob in zip(documents, topics, probs):
    topic_prob = prob[topic]  # -1 indicates an outlier topic
    topic_mapping.append({
        'text': doc,
        'topic': topic,
        'probability': topic_prob
    })

# Sort by topic for easier viewing (optional)
#topic_mapping = sorted(topic_mapping, key=lambda x: x['topic'])

# Print or analyze the mapping
for entry in topic_mapping:
    print(f"Text: {entry['text'][:100]}...")  # Print the first 100 characters for brevity
    print(f"Assigned Topic: {entry['topic']}, Probability: {entry['probability']}\n")


In [ ]:

with open("output.txt", "w") as file:
    file.write("\n".join([str(item) for item in topics]))

In [ ]:
print(len(topics))

22496


In [ ]:
print(len(documents))

22496


In [ ]:
import csv

with open("output.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Optionally, write headers if needed
    writer.writerow(["Document", "Topic"])

    for item1, item2 in zip(documents, topics):
        writer.writerow([item1, item2])


In [ ]:
topic_mapping

[{'text': '\ufeffTparti pris pierre maheu jean marc piotte yvon dionnendré brochu',
  'topic': -1,
  'probability': 0}]

In [ ]:
# Retrieve the top 15 words and their probabilities for each topic
topic_data = {}
for topic in unique_topics:
    words_scores = model.get_topic(topic)
    top_data = words_scores[:15]
    topic_data[topic] = top_data

# Save to CSV
csv_path = "/content/drive/My Drive/topic_words_probs.csv"  # Adjust path as needed
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Topic', 'Word', 'Probability'])
    for topic, data in topic_data.items():
        for word, prob in data:
            writer.writerow([topic, word, prob])

print(f"Top 15 words and their probabilities for each topic saved to {csv_path}")



Top 15 words and their probabilities for each topic saved to /content/drive/My Drive/topic_words_probs.csv


In [ ]:
sample_topic = unique_topics[0]
print(model.get_topic(sample_topic))


[('numéraire', 0.3055051411409662), ('envoi', 0.29138970606392156), ('autorisé', 0.29138970606392156), ('affranchissement', 0.26886635180423707), ('postes', 0.20598375493146726), ('publication', 0.19647141551233224), ('ministère', 0.17852497895519076), ('deuxième', 0.1624406037087906), ('présente', 0.15745948144969432), ('ottawa', 0.13260258127241778)]


In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import os

# Ensure the destination directory exists
destination_directory = os.getcwd() + "/wordclouds/"
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Function to generate and save a word cloud for a topic
def generate_and_save_word_cloud(topic_data, topic_num):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(topic_data))

    # Save the word cloud to a file
    filepath = os.path.join(destination_directory, f"topic_{topic_num}.png")
    wordcloud.to_file(filepath)
    print(f"Word Cloud for Topic {topic_num} saved to {filepath}")

# Generate and save word clouds for each topic
for topic, data in topic_data.items():
    generate_and_save_word_cloud(data, topic)



Word Cloud for Topic 0 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_0.png
Word Cloud for Topic 1 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_1.png
Word Cloud for Topic 2 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_2.png
Word Cloud for Topic 3 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_3.png
Word Cloud for Topic 4 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_4.png
Word Cloud for Topic 5 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_5.png
Word Cloud for Topic 6 saved to /content/drive/MyDrive/CESTA Project Belatedly Canceled/Topic modeling Parti Pris/wordclouds/topic_6.png
Word Cloud for Topic 7 saved to /content/